# post-processing

Scratch code to post-process `analysis.py` log files.

**TODO:**
- automate post-processing to compare centralized vs decentralized
- consider logging stuff as dicts with eval instead of lists
- move to hardware

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import decentralized as dec

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport decentralized.util

In [ ]:
%matplotlib ipympl
plt.rcParams['axes.grid'] = True

In [ ]:
fig = plt.figure()

In [ ]:
filename = dec.repopath / "logs" / "dec-mc-2_08-03-22_21.46.15.csv"
df = pd.read_csv(filename)

# Turn string lists into more usable objects.
df.loc[:, 'ids'] = df.loc[:, 'ids'].apply(eval)
df.loc[:, 'times'] = df.loc[:, 'times'].apply(lambda e: np.array(eval(e)))
df.loc[:, 'subgraphs'] = df.loc[:, 'subgraphs'].apply(eval)
df.loc[:, 'sizes'] = df.loc[:, 'subgraphs'].apply(lambda s: [len(si) for si in s])

In [ ]:
models = df.dynamics.unique().tolist()
n_agents_iter = df.n_agents.unique().tolist()

all_times = []
for model in models:
    model_times = []
    for n_agents in n_agents_iter[:-1]:
        frame = df.loc[(df.dynamics == model) & (df.n_agents == n_agents)]
        times_cen = frame.loc[frame.centralized == True].times
        times_dec = frame.loc[frame.centralized == False].times
        model_times.extend([np.vstack(times_cen).flatten(), np.vstack(times_dec).flatten()])
    all_times.append(model_times)

In [ ]:
plt.clf()
tick_locs = np.r_[1.5:n_agents_iter[-1]+2:2]
for i, (model, times) in enumerate(zip(models, all_times)):
    ax = fig.add_subplot(3, 1, i+1)
    bp = ax.boxplot(times, sym="kx", patch_artist=True)
    
    for patch, color in zip(bp['boxes'], plt.cm.Accent.colors):
        patch.set_facecolor(color)

    
    
plt.draw()

In [ ]:
test_df = df.loc[(df.dynamics == "DoubleIntDynamics4D") & (df.n_agents == 3)]
test_cen = test_df.loc[test_df.centralized == True]
test_dec = test_df.loc[test_df.centralized == False]
times = [np.vstack(test_cen.times).flatten(), np.vstack(test_dec.times).flatten()]

In [ ]:
plt.clf()
ax = plt.gca()
# ax.scatter(np.zeros_like(times_cen), times_cen)
# ax.scatter(np.ones_like(times_dec), times_dec)
# ax.violinplot(times, positions=[0, 1], showmeans=True, showextrema=True)
bp = ax.boxplot(times, sym="kx", labels=["centralized", "decentralized"], patch_artist=True)
plt.draw()

for patch, color in zip(bp['boxes'], ['pink', 'lightblue']):
    patch.set_facecolor(color)